## ✫faissによる近似近傍探索処理

In [1]:
# !pip install faiss-gpu

In [2]:
# GPUによる処理を前提
import faiss

res = faiss.StandardGpuResources()
dim = 768
nlist = 256
M = 32
nbits = 8
metric = faiss.METRIC_L2
ivfpq_config = faiss.GpuIndexIVFPQConfig()
ivfpq_config.usePrecomputedTables = True
index = faiss.GpuIndexIVFPQ(res, dim, nlist, M, nbits, metric, ivfpq_config)

In [3]:
# 前のステップまでに作成した（文章ID, 文章ベクトル）を配列としたリストをロードする
import pickle

with open('livedoor_vec_list.pkl', 'rb') as f:
    livedoor_vec_list = pickle.load(f)

In [4]:
len(livedoor_vec_list)
# 7376

7376

In [5]:
# faissによる学習データの準備
import numpy as np
import random
assert not index.is_trained
train_data = [v for _, v in livedoor_vec_list]
train_data = np.array(train_data, dtype=np.float32)

In [6]:
train_data.shape
# (7376, 768)

(7376, 768)

In [7]:
train_data = np.concatenate([train_data, train_data])

In [8]:
index.train(train_data)
assert index.is_trained

In [9]:
batch_size = 10000
for i in range(0, len(livedoor_vec_list), batch_size):
    input_vecs = []
    input_ids = []
    for item_id, vec in livedoor_vec_list[i:i+batch_size]:
        input_vecs.append(vec)
        input_ids.append(item_id)
    input_vecs = np.array(input_vecs, dtype=np.float32)
    input_ids = np.array(input_ids, dtype=np.int64)
    index.add_with_ids(input_vecs, input_ids)

# 学習済みインデックスの保存（GPUからCPUへ戻す）
converted_index_for_cpu = faiss.index_gpu_to_cpu(index)
faiss.write_index(converted_index_for_cpu, 'livedoor.index')

In [10]:
type(index)

faiss.swigfaiss.GpuIndexIVFPQ

### 本番利用時には保存されたindexからロードする処理から開始

In [11]:
import faiss
cpu_index = faiss.read_index('livedoor.index')
index = faiss.index_cpu_to_all_gpus(cpu_index)

In [12]:
# 文章ベクトル → 文章本文の辞書
import pickle

with open('livedoor_vec_to_text_dict.pkl', 'rb') as f:
    livedoor_vec_to_text_dict = pickle.load(f)

In [13]:
# 文章ベクトルリスト
import pickle
with open('livedoor_vec_list.pkl', 'rb') as f:
    livedoor_vec_list = pickle.load(f)

In [14]:
# 文章本文 → 文章ラベルの辞書
import pickle

with open('livedoor_text_to_label_dict.pkl', 'rb') as f:
    livedoor_text_to_label_dict = pickle.load(f)

### 検索用のベクトル変換

In [15]:
import pandas as pd
df = pd.read_csv('./triplet_test.tsv', sep='\t')
# df.head(3)
test_text = str(df['anchor'][6])

In [16]:
print(test_text)

全世界で6,500万部を売り上げた大ヒットミステリー小説を、『セブン』のデヴィッド・フィンチャー監督が映画化し、本年度アカデミー賞では主演女優賞をはじめ5部門でノミネートされた注目作『ドラゴン・タトゥーの女』。2月10日の公開に先駆けて、30日にはジャパンプレミアが行われ、翌31日にはデヴィッド・フィンチャー監督と主演ルーニー・マーラの来日記者会見が行われた。——まずは一言ご挨拶を。デヴィッド・フィンチャー：今日はお越しいただきありがとう。昨日はジャパンプレミアで、客席から舞台まで、長いランウェイを歩いてモデルのような気分でした。モデルは久しぶりなので、少し緊張しました（笑）。ルーニー・マーラ：初めての来日になります。まだあまり見れていないですが、東京はとっても好きです。午後少し時間があるので、街に出てみようと思います。日本に来れたことだけで嬉しいです。——大ベストセラー、さらに一度映画化されている作品を取り上げた理由は？スウェーデン版との違いは？デヴィッド・フィンチャー：スウェーデン版は一度しか観ていません。特に入念に観た訳ではないのですが、、強いて言うならば脚本がだいぶ違うと聞いています。何よりも自分が実際に原作を読んだときに感じたことを忠実に描くように心がけました。あえてスウェーデン版との違いをつくろうとして作った訳ではないんです。——リスベット役を引き受けた理由は？彼女のどこに共感しましたか？ルーニー・マーラ：原作を読んで好きになりました。読まれた方は皆さん彼女に共感を覚え好きになると思います。彼女をどのように演じたらいいかよく考えて、私はリスベットを演じことができる・理解できている、と思ったんです。彼女には色んなかたちで共感しました。人生の中で、自分が誤解されている、除け者にされている、と感じる経験は誰にでもあることだと思います。特にその点で共感することができました。引き受けた理由は、若い女優にとってこのような役に巡り合えるのはめったにないことですし、これは大きなチャンスだと思ったからです。——リスベットのキャラクター造形について、原作より魅力的に感じました。どのように独自性を出しましたか？デヴィッド・フィンチャー：特に足したことはありません。原作の中でかなり入念に描かれています。なので2作目、3作目も読んで創っていきました。映画ではキャラクターが何を考え

In [17]:
import sentence_transformers
import numpy as np
from sentence_transformers import SentenceTransformer

sbert = SentenceTransformer('./sbert')
vector = sbert.encode(test_text)
# queries = np.array([vector], dtype=np.float32)
# queries.shape
# (1, 768)

In [18]:
import faiss
import numpy as np

vecs = np.array([vector], dtype=np.float32)
d, i = index.search(vecs, 10) # 近傍ベクトルを10個とってくる
d = d[0]
i = i[0]
for item_id, distance in zip(i, d):
    # 文章IDと距離を表示
    print('文章ID:', item_id, '距離:', distance)
    n = np.array(livedoor_vec_list[item_id][1], dtype=np.float32)
    n = tuple(n)
    
    # 文章IDから文章を参照
    print(livedoor_vec_to_text_dict[n])
    print(livedoor_text_to_label_dict[livedoor_vec_to_text_dict[n]])
    print('---')

文章ID: 3466 距離: 4.4692826
全世界で6,500万部を売り上げた大ヒットミステリー小説を、『セブン』のデヴィッド・フィンチャー監督が映画化し、本年度アカデミー賞では主演女優賞をはじめ5部門でノミネートされた注目作『ドラゴン・タトゥーの女』。2月10日の公開に先駆けて、30日にはジャパンプレミアが行われ、翌31日にはデヴィッド・フィンチャー監督と主演ルーニー・マーラの来日記者会見が行われた。——まずは一言ご挨拶を。デヴィッド・フィンチャー：今日はお越しいただきありがとう。昨日はジャパンプレミアで、客席から舞台まで、長いランウェイを歩いてモデルのような気分でした。モデルは久しぶりなので、少し緊張しました（笑）。ルーニー・マーラ：初めての来日になります。まだあまり見れていないですが、東京はとっても好きです。午後少し時間があるので、街に出てみようと思います。日本に来れたことだけで嬉しいです。——大ベストセラー、さらに一度映画化されている作品を取り上げた理由は？スウェーデン版との違いは？デヴィッド・フィンチャー：スウェーデン版は一度しか観ていません。特に入念に観た訳ではないのですが、、強いて言うならば脚本がだいぶ違うと聞いています。何よりも自分が実際に原作を読んだときに感じたことを忠実に描くように心がけました。あえてスウェーデン版との違いをつくろうとして作った訳ではないんです。——リスベット役を引き受けた理由は？彼女のどこに共感しましたか？ルーニー・マーラ：原作を読んで好きになりました。読まれた方は皆さん彼女に共感を覚え好きになると思います。彼女をどのように演じたらいいかよく考えて、私はリスベットを演じことができる・理解できている、と思ったんです。彼女には色んなかたちで共感しました。人生の中で、自分が誤解されている、除け者にされている、と感じる経験は誰にでもあることだと思います。特にその点で共感することができました。引き受けた理由は、若い女優にとってこのような役に巡り合えるのはめったにないことですし、これは大きなチャンスだと思ったからです。——リスベットのキャラクター造形について、原作より魅力的に感じました。どのように独自性を出しましたか？デヴィッド・フィンチャー：特に足したことはありません。原作の中でかなり入念に描かれています。なので2作目、3作目も読ん